# Type charts

By **Franklin Oliveira**

-----
This notebook contains all code necessary to make the "type" charts from `repteis` database. Here you'll find some basic data treatment and charts' code. 

Database: <font color='blue'>'Compilacao Livros Repteis - 2 a 10 - 2020_04_28.xls'</font>.

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# pacotes para visualização rápida
import seaborn as sns
import matplotlib.pyplot as plt

# pacote para visualização principal
import altair as alt

# habilitando renderizador para notebook
# alt.renderers.enable('notebook')
# alt.renderers.enable('default')


# desabilitando limite de linhas
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [2]:
NewTable = pd.read_csv('./data/treated_db.csv', sep=';', encoding='utf-8-sig')

/home/franklin/anaconda3/envs/viz/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<br>

<font size=5>**Paleta de cores por Ordem**</font>

Abaixo está a imagem usada como inspiração (https://color.adobe.com/create/image)

<img src="./src/paleta_cores.jpeg" width='500px'>

Cores: 

- verde_escuro: #284021
- verde_claro: #88BF11
- amarelo: #D9CB0B
- laranja: #D99311
- laranja_escuro: #BF4417
- marrom-_laro: #BF8D7A

In [3]:
# p.s.: caudata is an error and should be removed. 
cores_ordem = {
    'Squamata': '#BF4417',
    'Testudines': '#D9CB0B', 
    'Crocodylia': '#284021'
}

ordens = list(cores_ordem.keys())
cores = list(cores_ordem.values())

**Paleta de Cores - Família:**

- grupo 1: 1 cor  (verde escuro)
<ul>
    <li style="color:#142611"><b>centroide 1</b></li>
</ul>
- grupo 2: 8 cores
<ul>
    <li style="color:#85D907"><b>centroide 2</b></li>
</ul>

['#d7ff81', '#bafd62', '#9feb3f', '#85d907', '#6cc700', '#52b700', '#35a600', '#0b9700', '#008800']

<font color="#d7ff81"><b>cor 1</b> (ficou fora)</font>
<font color="#bafd62"><b>cor 2</b></font>
<font color="#9feb3f"><b>cor 3</b></font>
<font color="#85d907"><b>cor 4</b></font>
<font color="#6cc700"><b>cor 5</b></font>
<font color="#52b700"><b>cor 6</b></font>
<font color="#35a600"><b>cor 7</b></font>
<font color="#0b9700"><b>cor 8</b></font>
<font color="#008800"><b>cor 9</b></font>


- grupo 3: 2 cores
#888C03
<ul>
    <li style="color:#22401E"><b>centroide 3 (puxando para tons frios mais claros)</b></li>
</ul>

<font color="#99b6b2"><b>cor 1</b></font>
<font color="#81a58b"><b>cor 2</b></font>


- grupo 4: 1 cor  (amarelo)
<ul>
    <li style="color:#F2CB07"><b>centroide 4</b></li>
</ul>

- grupo 5: 10 cores
<ul>
    <li style="color:#cb97d4"><b>centroide 5 (puxando para o roxo)</b></li>
</ul>

['#f8dcf9', '#ebc5ed', '#ddafe2', '#ce9ad6', '#bf86cc', '#af73c2', '#a160b8', '#924fae', '#833fa4'] #803da1

<font color="#f8dcf9"><b>cor 1</b></font>
<font color="#ebc5ed"><b>cor 2</b></font>
<font color="#ddafe2"><b>cor 3</b></font>
<font color="#ce9ad6"><b>cor 4</b></font>
<font color="#bf86cc"><b>cor 5</b></font>
<font color="#af73c2"><b>cor 6</b></font>
<font color="#a160b8"><b>cor 7</b></font>
<font color="#924fae"><b>cor 8</b></font>
<font color="#833fa4"><b>cor 9</b></font>
<font color="#803da1"><b>cor 10</b></font>


- grupo 6: 12 cores
<ul>
    <li style="color:#91F2E9"><b>centroide 6</b></li>
</ul>

['#c9fff9', '#b3eff2', '#9cdcea', '#83c9e2', '#68b7da', '#4aa6d2', '#2096ca', '#0087c1', '#0079b7']

['#cee5d8', '#b3d2d1', '#9bbfc9', '#83adc2', '#6d9bba', '#568ab2', '#3e7baa', '#226ca2', '#005e98']


<font color="#c9fff9"><b>cor 1</b></font>
<font color="#b3eff2"><b>cor 2</b></font>
<font color="#9cdcea"><b>cor 3</b></font>
<font color="#83c9e2"><b>cor 4</b></font>
<font color="#68b7da"><b>cor 5</b></font>
<font color="#4aa6d2"><b>cor 6</b></font>
<font color="#2096ca"><b>cor 7</b></font>
<font color="#0087c1"><b>cor 8</b></font>
<font color="#0079b7"><b>cor 9</b></font>
<font color="#3e7baa"><b>cor 10</b></font>
<font color="#226ca2"><b>cor 11</b></font>
<font color="#005e98"><b>cor 12</b></font>


- grupo 7: 3 cores
<ul>
    <li style="color:#8C1A0F"><b>centroide 7 (puxando para o marrom)</b></li>
</ul>

['#fde5bf', '#efd09f', '#e1bb82', '#d3a767', '#c6934d', '#b98033', '#ac6f18', '#9e5e00', '#914e00']

<font color="#ac6f18"><b>cor 1</b></font>
<font color="#9e5e00"><b>cor 2</b></font>
<font color="#914e00"><b>cor 3</b></font>


- grupo 8: 13 cores
<ul>
    <li style="color:#D9430D"><b>centroide 8</b></li>
</ul>

['#ffce9f', '#ffb683', '#ff9f69', '#ff8851', '#f5723b', '#e75b25', '#d9430d', '#cb2800', '#bc0000']

['#ff8f68', '#ff7e56', '#ff6b40', '#ee5829', '#d9430d', '#c62f00', '#b41b00', '#a40300', '#930000']

<font color="#ffce9f"><b>cor 1</b></font>
<font color="#ffb683"><b>cor 2</b></font>
<font color="#ff9f69"><b>cor 3</b></font>
<font color="#ff8851"><b>cor 4</b></font>
<font color="#f5723b"><b>cor 5</b></font>
<font color="#e75b25"><b>cor 6</b></font>
<font color="#d9430d"><b>cor 7</b></font>
<font color="#cb2800"><b>cor 8</b></font>
<font color="#bc0000"><b>cor 9</b></font>
<font color="#c62f00"><b>cor 10</b></font>
<font color="#b41b00"><b>cor 11</b></font>
<font color="#a40300"><b>cor 12</b></font>
<font color="#930000"><b>cor 13</b></font>

<br>

**TOTAL: 50 cores**

In [4]:
cores_familia = {
    # known errors treatment
#     '#n/d':'#000000',
#     'nan':'#000000',
    # grupo 1: Crocodylia
    'Alligatoridae':'#142611',
    # grupo 2: Testudines - Cryptodira
    'Cheloniidae':'#bafd62',
    'Chelydridae':'#9feb3f',
    'Dermochelyidae':'#85d907',
    'Emydidae':'#6cc700',
    'Geoemydidae':'#52b700',
    'Kinosternidae':'#35a600',
    'Testudinidae':'#0b9700',
    'Trionychidae':'#008800',
    # grupo 3: Testudines - Pleurodira
    'Chelidae':'#99b6b2' ,
    'Podocnemididae':'#81a58b',
    # grupo 4: Amphisbaenia - Amphisbaenia
    'Amphisbaenidae':'#F2CB07',
    # grupo 5: Sauria - Iguania
    'Agamidae':'#f8dcf9',
    'Chamaeleonidae':'#ebc5ed',
    'Dactyloidae':'#ddafe2',
    'Iguanidae':'#ce9ad6',
    'Hoplocercidae':'#bf86cc',
    'Leiosauridae':'#af73c2',
    'Liolaemidae':'#a160b8', 
    'Phrynosomatidae':'#924fae',
    'Polychrotidae':'#833fa4',
    'Tropiduridae':'#803da1',
    # grupo 6: Sauria - Scleroglossa
    'Anguidae':'#c9fff9',
    'Gekkonidae':'#b3eff2',
    'Gymnophthalmidae':'#9cdcea',
    'Helodermatidae':'#83c9e2',
    'Lacertidae':'#68b7da',
    'Mabuyinae':'#4aa6d2',
    'Phyllodactylidae':'#2096ca',
    'Scincidae':'#0087c1',
    'Sphaerodactylidae':'#0079b7',
    'Teiidae':'#3e7baa',
    'Varanidae':'#226ca2',
    'Xantusiidae':'#005e98',
    # grupo 7: Serpentes - Scolecophidia
    'Anomalepididae':'#ac6f18',
    'Leptotyphlopidae':'#9e5e00',
    'Typhlopidae':'#914e00', 
    # grupo 8: Alethinophidia
    'Aniliidae':'#ffce9f',
    'Boidae':'#ffb683',
    'Colubridae':'#ff9f69',
    'Dipsadidae':'#ff8851',
    'Elapidae':'#f5723b',
    'Homalopsidae':'#e75b25',
    'Lamprophiidae':'#d9430d', 
    'Loxocemidae':'#cb2800',
    'Natricidae':'#bc0000',
    'Pythonidae':'#c62f00',
    'Tropidophiidae':'#b41b00',
    'Viperidae':'#a40300',
    'Xenopeltidae':'#930000'
}

<br>


## Graphs

---

### Types (*per year*) per genus

x: Species1, cor: Type Status1, size: counts

In [5]:
# type info is in this column
NewTable['type_status'].value_counts(dropna= False)

NaN          22731
Parátipo       358
Holótipo        32
2005             5
Síntipo          4
Topótipo         1
Lectótipo        1
Name: type_status, dtype: int64

In [6]:
# subsetting
teste = NewTable[['altitude','familia','ordem', 'ano_descricao', 'qualificador_atual', 'numero_catalogo', 
                  'genero_atual', 'especie_atual', 'subespecie_atual', 'type_status']].copy()

# grouping by type, year and order
temp = teste.groupby(['type_status','ano_descricao', 'ordem']).count()['familia'].reset_index().rename(columns={
    'familia':'counts'
})

# p.s.: Cótipo and Topótipo are not types
temp = temp[(temp['type_status'] != 'Cótipo') & (temp['type_status'] != 'Topótipo')]

243 info. de tipos

### Gráf. de Tipos

In [7]:
tipo = alt.Chart(temp, height=150, title='Types per year').mark_circle().encode(
    x = alt.X('ano_descricao:O', title='description year'),
    y = alt.Y('type_status:N', title= 'type',
              sort=alt.EncodingSortField('tipo', op='count', order='descending')),
    color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores), title='order'), 
    size= alt.Size('counts', scale=alt.Scale(range=[10,600])),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    tooltip= [alt.Tooltip('type_status', title='type'),
              alt.Tooltip('ano_descricao', title='description year'),
              alt.Tooltip('counts', title='counts')]
)

tipo = tipo.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# tipo.save('./graphs/tipo/tipos_por_ano.html')
tipo

alt.Chart(...)

### tipos por ano (cores por FAMILIA)

In [8]:
# subsetting
teste = NewTable[['altitude','familia','ordem', 'ano_descricao', 'qualificador_atual', 'numero_catalogo', 
                  'genero_atual', 'especie_atual', 'subespecie_atual', 'type_status']].copy()

# grouping by type, year and order
temp = teste.groupby(['type_status','ano_descricao', 'familia']).count()['ordem'].reset_index().rename(columns={
    'ordem':'counts'
})

# p.s.: Cótipo and Topótipo are not types
temp = temp[(temp['type_status'] != 'Cótipo') & (temp['type_status'] != 'Topótipo')]

In [16]:
tipo = alt.Chart(temp, height=150,width=500, title='Types per year').mark_circle().encode(
    x = alt.X('ano_descricao:O', title='description year'),
    y = alt.Y('type_status:N', title= 'type',
              sort=alt.EncodingSortField('tipo', op='count', order='descending')),
    color= alt.Color('familia:N', scale=alt.Scale(domain=list(cores_familia.keys()), 
                                                  range=list(cores_familia.values())), title='Family',
                    legend= alt.Legend(columns=4, symbolLimit=50)), 
    size= alt.Size('counts', scale=alt.Scale(range=[10,600])),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    tooltip= [alt.Tooltip('type_status', title='type'),
              alt.Tooltip('ano_descricao', title='description year'),
              alt.Tooltip('counts', title='counts')]
)

tipo = tipo.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# tipo.save('./graphs/tipo/tipos_por_ano-FAMILIA.html')
# tipo

<br>

#### defining some parameters

<font color='red'>**OBS:** essas cores foram definidas quando fizemos esses gráficos para a coleção de crustáceas (faz tempo). Talvez precisem ser atualizadas </font>

**colors:**
<ul>
    <li style='color:#d62728;'><b> #d62728 </b> - vermelho 1</li>
    <li style='color:#d62729;'><b> #d62729 </b> - vermelho 2</li>
    <li style='color:#f58518;'><b> #f58518 </b> - laranja 1</li>
    <li style='color:#f58519;'><b> #f58519 </b> - laranja 2</li>
    <li style='color:#d95f02;'><b> #d95f02 </b> - laranja escuro</li>
    <li style='color:#4daf4a;'><b> #4daf4a </b> - verde</li>
    <li style='color:#8c6d31;'><b> #8c6d31 </b> - marrom</li>
    <li style='color:#79706e;'><b> #79706e </b> - azul</li>
    <li style='color:#bab0ac;'><b> #bab0ac </b> - azul claro</li>
    <li style='color:#d8b5a5;'><b> #d8b5a5 </b> - bege</li>
    <li style='color:#000000;'><b> #000000 </b> - preto</li>
</ul>

In [7]:
NewTable['type_status'] = NewTable['type_status'].apply(lambda x:str(x))
types = NewTable['type_status'].apply(lambda x:str(x)).unique()

In [8]:
types

array(['nan', 'Holótipo', 'Parátipo', 'Topótipo', 'Lectótipo', 'Síntipo',
       '2005'], dtype=object)

In [9]:
# dictionary with type:color
type_color = {
    'nan':'#000000',      # preto
    'Holótipo':'#d62728',  # vermelho 1
    'Síntipo':'#d62729',   # vermelho 2
    'Parátipo':'#4daf4a',  # verde
    'Topótipo':'#8c6d31',  # marrom
    'Lectótipo':'#f58518', # laranja 1
    '2005':'#79706e',      # azul  OBS: ERRO NA BASE!
}

<br>

## Ano de coleta de tipos x ano de descrição

In [10]:
temp = NewTable[NewTable['type_status'] != 'nan']

In [11]:
g1 = alt.Chart(temp, width=500, height=500).mark_circle().encode(
    x = alt.X('ano_descricao:O', title='Ano de descrição'),
    y = alt.Y('ano_coleta:O', title='Ano de coleta', 
             sort= alt.EncodingSortField('ano_coleta', op='min', order='descending')),
    color = alt.Color('type_status:N', title='Tipo', 
                     scale= alt.Scale(domain= list(type_color.keys()), range= list(type_color.values()))),
    tooltip = alt.Tooltip(['numero_catalogo', 'familia', 'ordem', 'type_status', 
                           'ano_coleta', 'ano_descricao'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

g1.save('./graphs/tipo/ano_coleta-vs-ano_descricao.html')

g1

alt.Chart(...)

<br>

### grouping and counting

In [12]:
counts = NewTable.groupby(['first_author', 'type_status', 'ano_descricao']).count()['class'].reset_index()
counts.rename(columns={'class':'counts'}, inplace=True)

## types and non-types per author

In [16]:
# AJUSTAR DEPOIS: preciso que a coluna de ano_descricao esteja limpa para conseguir ordenar!
# sort_determiners = list(counts[counts['type_status'] != 'nan'].groupby(['determinator_full_name']).min()['ano_determinacao'].reset_index(
# ).sort_values('ano_determinacao')['determinator_full_name'])

In [17]:
g1 = alt.Chart(counts[counts['type_status'] != 'nan'], title='Tipos por Autor', 
               height=300, width=400).mark_circle().encode(
    x = alt.X('ano_descricao:O', title='Ano de Descrição'),
    y = alt.Y('first_author:N', title=''),  # AJUSTAR DEPOIS
#              sort= sort_determiners),
    color = alt.Color('type_status:N', title='Tipo', 
                     scale = alt.Scale(domain=list(type_color.keys()), range= list(type_color.values()))), 
    size = alt.Size('counts:Q', scale= alt.Scale(range=[10,500])), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    tooltip = alt.Tooltip(['first_author', 'type_status', 'ano_descricao', 'counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# saving graph
# g1.save('./graphs/tipo/tipos_por_autor.html')

g1

alt.Chart(...)

**Não-tipos por Autor: faz sentido?**

In [18]:
# sort_determiners = list(counts[counts['type_status'] == 'nan'].groupby(['determinator_full_name']).min()['ano_determinacao'].reset_index(
# ).sort_values('ano_determinacao')['determinator_full_name'])

In [19]:
# g1 = alt.Chart(counts[counts['type_status'] == 'nan'], title='Não-tipos por Determinador', 
#                height=1000, width=400).mark_circle(color='#000000').encode(
#     x = alt.X('ano_determinacao:O', title='Ano de Determinação'),
#     y = alt.Y('determinator_full_name:N', title='', 
#              sort= sort_determiners),
#     size = alt.Size('counts:Q'), 
#     tooltip = alt.Tooltip(['determinator_full_name', 'ano_determinacao', 'counts'])
# )

# g1 = g1.configure_title(fontSize=16).configure_axis(
#     labelFontSize=12,
#     titleFontSize=12
# ).configure_legend(
#     labelFontSize=12,
#     titleFontSize=12
# )

# # saving graph
# g1.save('./graphs/tipo/nao-tipos_por_determinador.html')

# # g1

<br>

## types per collector

-----

### grouping and counting

In [20]:
counts = NewTable.groupby(['collector_full_name', 'type_status', 'ano_coleta']).count()['class'].reset_index()
counts.rename(columns={'class':'counts'}, inplace=True)
counts.sort_values('counts', inplace=True, ascending=False)

In [21]:
sort_determiners = list(counts[counts['type_status'] != 'nan'].groupby(['collector_full_name']).min()['ano_coleta'].reset_index(
).sort_values('ano_coleta')['collector_full_name'])

In [24]:
g1 = alt.Chart(counts[counts['type_status'] != 'nan'], title='Tipos por Coletor', 
               height=700, width=400).mark_circle().encode(
    x = alt.X('ano_coleta:O', title='Ano de Determinação'),
    y = alt.Y('collector_full_name:N', title='', 
             sort= sort_determiners),
    color = alt.Color('type_status:N', title='Tipo', 
                     scale = alt.Scale(domain=list(type_color.keys()), range= list(type_color.values()))), 
    size = alt.Size('counts:Q'), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    tooltip = alt.Tooltip(['collector_full_name', 'type_status', 'ano_coleta', 'counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# saving graph
g1.save('./graphs/tipo/tipos_por_coletor.html')

# g1

In [25]:
sort_determiners = list(counts[counts['type_status'] != 'nan'].groupby(['collector_full_name']).min()['ano_coleta'].reset_index(
).sort_values('ano_coleta')['collector_full_name'])

In [27]:
g1 = alt.Chart(counts[counts['type_status'] == 'nan'].iloc[0:200], title='Não-tipos por Coletor', 
               height=700, width=400).mark_circle(color= '#000000').encode(
    x = alt.X('ano_coleta:O', title='Ano de Determinação'),
    y = alt.Y('collector_full_name:N', title='', 
             sort= alt.EncodingSortField(field= 'counts', op='sum', order='descending')),
    size = alt.Size('counts:Q'), 
    tooltip = alt.Tooltip(['collector_full_name', 'type_status', 'ano_coleta', 'counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# saving graph
g1.save('./graphs/tipo/nao-tipos_por_coletor.html')

# g1

<br>

## Types per family

In [28]:
# grouping by type, year and order
temp = NewTable.groupby(['type_status','ano_descricao', 'familia', 'ordem']).count()['class'].reset_index().rename(columns={
    'class':'counts'
})

temp = temp[temp['type_status'] != 'nan'].copy()

In [29]:
# AJUSTAR DEPOIS: preciso que o ano_descricao esteja limpo para poder ordenar
# family_order = list(temp.groupby(['familia']).min(
#     )['ano_descricao'].reset_index().sort_values('ano_descricao')['familia'])

In [31]:
tipo = alt.Chart(temp, height=400, width=400, title='Types per Family').mark_point(filled=False).encode(
    x = alt.X('ano_descricao:O', title='Description Year'),
    y = alt.Y('familia:N', title= 'Family'),  # AJUSTAR DEPOIS! 
#               sort= family_order),
    color= alt.Color('ordem:N', title='Order',
                    scale= alt.Scale(domain=list(cores_ordem.keys()), range=list(cores_ordem.values()))), 
    size= alt.Size('counts', title='Counts', scale=alt.Scale(range=[10,500])),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    shape= alt.Shape('type_status:N', title='Type'), 
#                     scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
#                                      range=['triangle', 'square', 'circle'])),
#     opacity= alt.Opacity('type:N', scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
#                                                      range=[1, 0.5, 1])),
    tooltip= [alt.Tooltip('type_status', title='type'),
              alt.Tooltip('ano_descricao', title='description year'),
              alt.Tooltip('counts', title='counts')]
)

tipo = tipo.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

tipo.save('./graphs/tipo/tipos_por_familia-colorido-ORDEM.html')

tipo

alt.Chart(...)

In [37]:
familias = [f for f in cores_familia.keys() if f in temp['familia'].unique()]
cores_temp = [cores_familia[f] for f in familias]

tipo = alt.Chart(temp, height=400, width=400, title='Types per Family').mark_point(filled=False).encode(
    x = alt.X('ano_descricao:O', title='Description Year'),
    y = alt.Y('familia:N', title= 'Family'),  # AJUSTAR DEPOIS! 
#               sort= family_order),
    color= alt.Color('familia:N', title='Family',
                     legend = alt.Legend(columns=2),
                    scale= alt.Scale(domain=familias, range= cores_temp)), 
    size= alt.Size('counts', title='Counts', scale=alt.Scale(range=[10,500])),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    shape= alt.Shape('type_status:N', title='Type'), 
#                     scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
#                                      range=['triangle', 'square', 'circle'])),
#     opacity= alt.Opacity('type:N', scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
#                                                      range=[1, 0.5, 1])),
    tooltip= [alt.Tooltip('type_status', title='type'),
              alt.Tooltip('ano_descricao', title='description year'),
              alt.Tooltip('counts', title='counts')]
)

tipo = tipo.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# tipo.save('./graphs/tipo/tipos_por_familia.html')

tipo

alt.Chart(...)

## Types per Genus 

same graph as above, with gender on Y axis and colored by type

In [40]:
# grouping by type, year and order
temp = NewTable.groupby(['type_status','ano_descricao', 'genero_atual', 'familia']).count()['class'].reset_index().rename(columns={
    'class':'counts'
})

# p.s.: Cótipo and Topótipo are not types
temp = temp[(temp['type_status'] != 'nan')]

In [41]:
# AJUSTAR DEPOIS: preciso que a coluna ano_descricao esteja limpa para conseguir odenar!
# genus_order = list(temp.groupby(['genero_atual']).min()['ano_coleta'].reset_index().sort_values('ano_coleta')['genero_atual'])

In [45]:
familias = [f for f in cores_familia.keys() if f in temp['familia'].unique()]
cores_temp = [cores_familia[f] for f in familias]

tipo = alt.Chart(temp, height=400, width= 400, title='Types per Genus').mark_point(filled=False).encode(
    x = alt.X('ano_descricao:O', title='Description Year'),
    y = alt.Y('genero_atual:N', title= 'Genus'),  # AJUSTAR DEPOIS!
#               sort=genus_order),
    color= alt.Color('familia:N', title='Family',
                     legend= alt.Legend(columns=2, symbolLimit=50),
                    scale= alt.Scale(domain=familias, range= cores_temp)), 
    size= alt.Size('counts', title='Counts',scale=alt.Scale(range=[10,500])),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    shape= alt.Shape('type_status:N', title='Type'), 
#                     scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
#                                      range=['triangle', 'square', 'circle'])),
#     opacity= alt.Opacity('type:N', scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype'],
#                                                      range=[1, 0.5, 1])),
    tooltip= [alt.Tooltip('type_status', title='type'),
              alt.Tooltip('ano_descricao', title='description year'),
              alt.Tooltip('counts', title='counts')]
)

tipo = tipo.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

tipo.save('./graphs/tipo/tipos_por_genero-primeiro_ano.html')

tipo

alt.Chart(...)

<br>

**The end!**

-----